In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [16]:
data = pd.read_csv('../dataset/EURUSDDaily.csv')
data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Unknown']
data = data.drop(['Date', 'Unknown'], axis=1)

data.head()

,Open,High,Low,Close,Volume
0,1.2844,1.2874,1.2835,1.2871,511
1,1.2872,1.2876,1.2742,1.2758,1541
2,1.2757,1.2832,1.2748,1.2815,1101
3,1.2813,1.2887,1.2793,1.2796,1711
4,1.2795,1.2848,1.2737,1.2769,1961


In [17]:
training = data[:5600].copy()
training

,Open,High,Low,Close,Volume
0,1.28440,1.28740,1.28350,1.28710,511
1,1.28720,1.28760,1.27420,1.27580,1541
2,1.27570,1.28320,1.27480,1.28150,1101
3,1.28130,1.28870,1.27930,1.27960,1711
4,1.27950,1.28480,1.27370,1.27690,1961
...,...,...,...,...,...
5595,1.18126,1.18367,1.16403,1.16503,89540
5596,1.16503,1.16568,1.15742,1.16075,92165
5597,1.16054,1.16578,1.15933,1.16505,59230
5598,1.16503,1.16612,1.16248,1.16457,57679


In [18]:
testing = data[5600:]
testing

,Open,High,Low,Close,Volume
5600,1.16177,1.16782,1.16130,1.16559,79755
5601,1.16559,1.16906,1.15991,1.16066,69187
5602,1.16097,1.16241,1.15802,1.16089,66254
5603,1.16091,1.16157,1.15540,1.15854,73309
5604,1.15854,1.16110,1.15792,1.15943,59168
...,...,...,...,...,...
6219,1.08797,1.10586,1.08700,1.10291,149438
6220,1.10290,1.11470,1.09533,1.11431,139186
6221,1.11159,1.11439,1.10099,1.10427,109850
6222,1.10414,1.10537,1.09266,1.10349,132367


In [37]:
scaler = MinMaxScaler()
training = scaler.fit_transform(training)
training

array([[0.59199793, 0.58973029, 0.60322032, 0.59560155, 0.00171112],
       [0.59562346, 0.58998963, 0.59104595, 0.58098318, 0.0051669 ],
       [0.58073288, 0.58428423, 0.59183139, 0.58835705, 0.00369064],
       ...,
       [0.43161984, 0.43202801, 0.44067286, 0.43771022, 0.19872102],
       [0.43743364, 0.43246888, 0.44479644, 0.43708926, 0.19351722],
       [0.43676033, 0.43196317, 0.44237466, 0.43355757, 0.23971401]])

In [20]:
X_train = []
y_train = []

for i in range(60, training.shape[0]):
    X_train.append(training[i-60:i])
    y_train.append(training[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

In [21]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

regressor = Sequential()

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 5)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 120, activation = 'relu'))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

In [22]:
regressor.compile(optimizer='adam', loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs=1, batch_size=32)

174/174 [==============================] - 41s 238ms/step - loss: 0.0138


In [44]:
past_60_days = pd.DataFrame(data=training[-60:].flatten())
df = past_60_days.append(testing, ignore_index = False)
df.head()

,0,Close,High,Low,Open,Volume
0,0.451094,NaN,NaN,NaN,NaN,NaN
1,0.448522,NaN,NaN,NaN,NaN,NaN
2,0.455151,NaN,NaN,NaN,NaN,NaN
3,0.453493,NaN,NaN,NaN,NaN,NaN
4,0.355251,NaN,NaN,NaN,NaN,NaN


In [13]:
inputs = scaler.transform(df)
inputs

NameError: name 'df' is not defined

In [ ]:
X_test = []
y_test = []

for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i])
    y_test.append(inputs[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
scaler.scale_